In [ ]:
"""
# 🔀 Hybrid Recommendation Model

Combine Content-Based and Collaborative Filtering
"""

# ## 1. Setup
import pandas as pd
import sys
sys.path.append('..')

from src.models.content_based import ContentBasedRecommender
from src.models.collaborative import CollaborativeFilteringRecommender
from src.models.hybrid_recommender import HybridRecommender
from config import *

# ## 2. Load Models
print("=== Loading Models ===")

# Content-Based
cb_model = ContentBasedRecommender()
cb_model.load_model(CONTENT_MODEL, TFIDF_VECTORIZER)
print("✅ Content-Based loaded")

# Collaborative
cf_model = CollaborativeFilteringRecommender()
cf_model.load_model(COLLABORATIVE_MODEL)
print("✅ Collaborative loaded")

# ## 3. Create Hybrid Model
print("\n=== Creating Hybrid Model ===")

hybrid_model = HybridRecommender(
    content_model=cb_model,
    collaborative_model=cf_model,
    content_weight=0.4,
    collaborative_weight=0.6,
    strategy='weighted'
)

# ## 4. Test Hybrid Recommendations
print("\n=== Testing Hybrid Model ===")

test_anime_id = 1535  # Death Note
recommendations = hybrid_model.recommend(anime_id=test_anime_id, top_n=10)

print(f"\nHybrid Recommendations for Death Note:")
print(recommendations[['Name', 'Score', 'cb_score', 'cf_score', 'hybrid_score']])

# ## 5. Compare Strategies
print("\n=== Comparing Strategies ===")

# Weighted
hybrid_model.strategy = 'weighted'
recs_weighted = hybrid_model.recommend(anime_id=test_anime_id, top_n=5)
print("\nWeighted Strategy:")
print(recs_weighted[['Name', 'hybrid_score']])

# Mixed
hybrid_model.strategy = 'mixed'
recs_mixed = hybrid_model.recommend(anime_id=test_anime_id, top_n=5)
print("\nMixed Strategy:")
print(recs_mixed[['Name']])

# ## 6. Visualize Scores
import matplotlib.pyplot as plt

hybrid_model.strategy = 'weighted'
recs = hybrid_model.recommend(anime_id=test_anime_id, top_n=10)

fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(recs))
width = 0.25

ax.bar([i - width for i in x], recs['cb_score'], width, label='Content-Based', alpha=0.8)
ax.bar(x, recs['cf_score'], width, label='Collaborative', alpha=0.8)
ax.bar([i + width for i in x], recs['hybrid_score'], width, label='Hybrid', alpha=0.8)

ax.set_xlabel('Anime')
ax.set_ylabel('Score')
ax.set_title('Model Score Comparison')
ax.set_xticks(x)
ax.set_xticklabels(recs['Name'], rotation=45, ha='right')
ax.legend()
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# ## 7. Save Hybrid Model
hybrid_model.save_model(HYBRID_MODEL)
print(f"\n✅ Hybrid model saved to {HYBRID_MODEL}")

# ## 8. Performance Analysis
print("\n=== Performance Analysis ===")

# Compare top-5 from each model
cb_recs = cb_model.recommend(test_anime_id, top_n=5)
cf_recs = cf_model.recommend_for_user(1, top_n=5)  # Example user
hybrid_recs = recs.head(5)

print("\nContent-Based Top 5:")
print(cb_recs['Name'].tolist())

print("\nCollaborative Top 5:")
print(cf_recs['Name'].tolist())

print("\nHybrid Top 5:")
print(hybrid_recs['Name'].tolist())

print("\n✅ Hybrid Model Complete!")